In [23]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D,BatchNormalization
import pickle
import keras


In [24]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [4]:
print(x_train.shape) # shape of cifar10 images

(50000, 32, 32, 3)


In [5]:
print(y_train.shape) #labels

(50000, 1)


In [6]:
print(x_test.shape)

(10000, 32, 32, 3)


In [25]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#makes dataset a type of gaussian distribution
mean = np.mean(x_train,axis=(0,1,2,3)) 
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)



In [29]:
model = Sequential() #makes layers  sequential
model.add(Conv2D(64,(3,3), input_shape=x_train.shape[1:],padding='same',kernel_regularizer = regularizers.l2(1e-4)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2))) #reduces dimensions of convolution output by taking max element from each cell
model.add(Dropout(0.2)) # randomly turnsoff some percent of nodes to reduce overfitting

model.add(Conv2D(64,(3,3),input_shape = x_train.shape[1:],padding='same',kernel_regularizer = regularizers.l2(1e-4)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),input_shape = x_train.shape[1:],padding='same',kernel_regularizer = regularizers.l2(1e-4)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten()) #makes data into single column
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

#data augmentation- flips images back and forth and learns from it
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

datagen.fit(x_train)

model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam",metrics=["accuracy"])

model.fit_generator(datagen.flow(x_train,y_train,batch_size=64),steps_per_epoch=x_train.shape[0]//64,verbose=1,validation_data=(x_test,y_test),epochs=10)
model.save_weights('cifar10_adam_regul.h5')



Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_45 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
activation_46 (Activation)   (None, 16, 16, 64)      

In [30]:
val_loss,val_acc = model.evaluate(x_test,y_test)
print(val_loss,val_acc)

10000/10000 [==============================] - 11s 1ms/sample - loss: 0.8261 - acc: 0.7259
0.8260829696655273 0.7259


In [ ]:
#References
#1. deep learning by sentdex on youtube
